In [ ]:
import math
import pandas as pd

In [ ]:
def format_mean_std(m, s):
    return "OOM" if math.isnan(m) else f"{m*1000:.1f} ± {s*1000:.3f} ms"

def combine_mean_std(df, mean_name, std_name, new_name):
    df[new_name] = df.apply(lambda row: format_mean_std(row[mean_name], row[std_name]), axis=1)
    df.drop(columns=[mean_name, std_name], inplace=True)

In [ ]:
results = pd.read_csv("raw_data/benchmark_results.csv")
fw = results[results.forward_only].drop(columns=["forward_only", "backward_mean", "backward_std", "use_optimizer"]).rename(columns={"forward_mean": "forward_only_mean", "forward_std": "forward_only_std"})
fw_bk = results[~results.forward_only].drop(columns=["forward_only", "use_optimizer"])
combined = fw.merge(fw_bk, how="left")

In [ ]:
combine_mean_std(combined, "forward_only_mean", "forward_only_std", "forward infer ($\\mu \\pm \\sigma$)")
combine_mean_std(combined, "forward_mean", "forward_std", "forward train ($\\mu \\pm \\sigma$)")
combine_mean_std(combined, "backward_mean", "backward_std", "backward  ($\\mu \\pm \\sigma$)")

In [ ]:
combined.rename(columns={"model.context_length": "context", "num_warmup_steps": "warmup"}, inplace=True)
combined["model"] = combined["model"].replace("'2.7B'", "2.7B")

In [ ]:
for model in combined.model.unique():
    for context in combined.context.unique():
        df = combined[(combined.model==model) & (combined.context == context)].drop(columns=["model", "context"])
        tex = df.to_latex(index=False, caption=f"{model} {context}", escape=False, bold_rows=True)
        tex = tex.replace(r"\begin{table}", r"\begin{table}[H]")
        print(tex)